# Let's do some biomechanics !!


Note: To run a cell, you press SHIFT+ENTER


## Load a previously created bioMod file
This first section prepares Python and load a `.bioMod` file as shown in the `ModelCreation` script.

In [ ]:
# So first, let's import all the required modules 
import biorbd  # This is the core that will do all the calculation

from copy import copy  # Allows to copy arrays
import numpy  # Numpy is a python module that helps dealing with the mathematics of matrices and vectors
numpy.set_printoptions(precision=4, suppress=True)  # Make the printing of numpy matrices prettier

from matplotlib import pyplot

In [ ]:
# Import and interpret the bioMod of the full body
model = biorbd.Model("models/SimpleBody.bioMod")

# Define some useful information
n_q = model.nbQ()  # The number of degrees of freedom in the model
arm_index = 3  # The index of the arm in the vector of degree of freedom. This is determined by the model

## Creating some data
The next section generates data. 
This is to simulate collecting data on a real human.
The movement we simulate here is simply raising the arm vertical point down to horizontal pointing forward

The strategy is to create a collection of position (stored in a matrix of "n degrees-of-freedom" by "n frames").
Then to replace some of the data points (namely those related to the arm) so it performs the required movement.
Finally, we will find the position of the skin markers for each of the data points (This is as if you were moving the avatar at a position, and the write down the 3d position of each blue dots). 

Remember that the arm in the SimpleBody.bioMod is the 4th degree-of-freedom so it is indexed "3" as the first index is "0".
The downward pointing is when $q[3] = 0.15$ radian and fronward point at $1.7$ radian. 
We simulate 100 frames in all


In [ ]:
# Let's declare some variables to make our life easy
number_frames = 100
frame_by_second = 100
starting_arm_position = 0.15
end_arm_position = 1.7
with_noise = True

# Get us a time vector which will be useful while graphing
time_vector = numpy.linspace(0, number_frames / frame_by_second, number_frames)

# First create a static body at each time frame
# So let's have the full body at a starting posture (arm down and knee flexed)
# Note: the transpose ensures that it is a column vector (time will be on each column)
full_body_position = numpy.array([[0, 0, -0.15, starting_arm_position, 0.7, -1, -1.1]]).transpose()

# Repeat that full body position, one for each frame
full_body_all_position = numpy.repeat(full_body_position, number_frames, axis=1)  # axis=1 is to repeat on columns

In [ ]:
# Now we can create an arm movement over time
arm_position_over_time = numpy.linspace(starting_arm_position, end_arm_position, number_frames)

# Add some noise as no data are perfect
if with_noise:
    noise = (numpy.random.rand(arm_position_over_time.shape[0]) - 0.5) / 20  # Noise of ±0.025
    arm_position_over_time_noised = arm_position_over_time + noise
else:
    arm_position_over_time_noised = arm_position_over_time

# And replace the arm position in the body positions 
full_body_all_position[arm_index, :] = arm_position_over_time_noised

In [ ]:
# If you want to see the movement you just simulated, you can run this cell
# Note: Due to Jupyter limitations, this may or may not cause issue
# Please also note that this cell will crash if runned from the binder

import bioviz

# Send the previously loaded model to the vizualizer
viz = bioviz.Viz(loaded_model=model)

# Move the model to a recognizable position (arm raised and knee flexed)
viz.load_movement(full_body_all_position)

# Halt the program so you can admire the movement with the vizualiser. Closing the window should allow to continue
viz.exec()

In [ ]:
# Finally we can "collect" the position of each skin markers at each frame. 
# This effectively simulates a data collection. Data could originate from 
# sofisticated equipment such as a Vicon system or it can be produced using
# more lay software such as Kinovea. The important bit is that the data are
# structured in a way that allows to subtract simulated data to real data
all_skin_markers = []
for f in range(number_frames):
    # Get the skin marker at a position
    skin_markers = list(model.markers(full_body_all_position[:, f]))
    
    # The next for loop goes over all the skin markers and transform them in vector 
    # This is mandatory because of the way biorbd stores data
    for i in range(len(skin_markers)):
        skin_markers[i] = skin_markers[i].to_array()
    
    # Put all these skin marker of a frame in a 3 by "n markers" matrix (3 for x, y and z components)
    skin_markers = numpy.array(skin_markers).transpose()
        
    # Store the results in the main data list
    all_skin_markers.append(skin_markers)

## Inverse kinematics
Inverse kinematics tries to find what are the positions that could have produced a set of given markers
In our case, these markers are the simulated one. 
In the real life, it would be collected data

The stategy is to simulate a lot a different position and find the one that reduces the most the error between the measured markers with the simulated one. Then to repeat for each frame.

In [ ]:
# First let's define a function that takes in a position (q) and spits out a matrix of markers (3 x nMarkers)
def forwardKinematics(q):
    markers = list(model.markers(q))
    
    # Put these value in a single vector by concatenating (stacking top/down)
    markersVector = numpy.ndarray((0,))
    for i in range(len(markers)):
        markersVector = numpy.concatenate((markersVector, markers[i].to_array()), axis=0)

    # Return the markers matrix to caller (newaxis converts the vector into a matrix with one column)
    return markersVector[:, numpy.newaxis]

# Define also a jacobian function
def jacobian(q):
    # Now test a bunch of positions to find the best (which minimizes the error with real markers)
    jacobian = list(model.markersJacobian(q))

    # Put these value in a matrix by concatenating (stacking top/down)
    jacobianMatrix = numpy.ndarray((0, n_q))
    for i in range(len(jacobian)):
        jacobianMatrix = numpy.concatenate((jacobianMatrix, jacobian[i].to_array()), axis=0)
    
    # Return the jacobian matrix to caller
    return jacobianMatrix

# Finally define the inverse kinematics function base on 
# https://en.wikipedia.org/wiki/Inverse_kinematics#The_Jacobian_inverse_technique
def inverseKinematics(real_markers, q_init, number_of_iterations=5):
    for i in range(number_of_iterations):
        # Get the marker and jacobian at q_init
        simulated_markers = forwardKinematics(q_init[:, 0])
        jacobianMatrix = jacobian(q_init[:, 0])

        # Compute the step length to perform. Reminder: @ is a matrix multiplication operator
        q_advance = numpy.linalg.pinv(jacobianMatrix) @ (simulated_markers - real_markers)

        # Advance the q_init and repeat until number_of_iterations is exhausted
        q_init -= q_advance
    return q_init

In [ ]:
# The let's test these functions and print the result to make sure it works as expected
q = numpy.array([0, 0, -0.15, 1.7, 0.7, -1, 0.45])
print(f"The markers when the body is at {q} is:")
print("")
print(forwardKinematics(q))
print("")
print("The jacobian matrix at the same position is:")
print(jacobian(q))

In [ ]:
# Perform the inverse kinematics for the frame 0
frame = 0
number_of_iterations = 5  # Number of Newton iterations

# First get the "collected" real markers data
real_markers = all_skin_markers[frame].transpose().flatten()[:, numpy.newaxis]

# Perform the inverse kinematics
q_init = numpy.zeros((n_q, 1))  # Initialize to a poor position (all zeros)
q_inverse = inverseKinematics(real_markers, q_init, number_of_iterations)
    
# Compare the results with the expected value (that we know because we created the data ourselves)
q_real = full_body_all_position[:, frame][:, numpy.newaxis]  # Get the reference answer

print(f"The expected answer is: {q_real.transpose()}")
print(f"The computed answer is: {q_inverse.transpose()}")
print(f"The RMSE between them is: {numpy.sqrt(sum((q_inverse - q_real)**2))}")

In [ ]:
# Now that we know how to do it for one frame, we can do it at each frame
# But that time, we can use a better q_init guess from the second frame as we know the 
# body won't move much between frames

number_of_iterations = 5  # Number of Newton iterations

q_init = numpy.zeros((n_q, 1))  # Initialize to a poor position (all zeros)
q_inverse = numpy.ndarray((n_q, number_frames))
for frame in range(len(all_skin_markers)):
    # First get the "collected" real markers data
    real_markers = all_skin_markers[frame].transpose().flatten()[:, numpy.newaxis]

    # Perform the inverse kinematics
    q_inverse[:, frame] = inverseKinematics(real_markers, q_init, number_of_iterations)[:, 0]
    
    # Take the previous position as the starting point. 
    q_init = copy(q_inverse[:, frame][:, numpy.newaxis])
    number_of_iterations = 2  # We don't need to perform as much iteration now that we have a good starting point

# Now print the sum RMSE
q_real = full_body_all_position  # Get the reference answer
print(f"The sum RMSE is: {sum(numpy.sqrt(sum((q_inverse - q_real)**2)))}")

In [ ]:
# We can finally graph the real kinematics of the arm against the reconstructed
pyplot.title("Arm angle against time")
pyplot.ylabel("Arm angle (rad)")
pyplot.xlabel("Time (s)")

pyplot.plot(time_vector, q_real[arm_index, :])
pyplot.plot(time_vector, q_inverse[arm_index, :])

In [ ]:
# Let's finally save the results so it can be used in the inverse dynamics script
import os
if not os.path.isdir("results"):
    os.mkdir("results")

numpy.save("results/inverse_kinematics.npy", q_inverse)